In [ ]:
import matplotlib.font_manager

system_fonts = matplotlib.font_manager.findSystemFonts(fontpaths=None, fontext='ttf')
indexes_to_remove = [17, 34, 44, 166, 174, 175, 178, 186, 263, 297, 317]
system_fonts = [font for i, font in enumerate(system_fonts) if i not in indexes_to_remove]

In [ ]:
from functions import (
    create_blank_pdf,
    find_optimal_placement,
    can_place_image,
    save_pdf, 
    place_image_in_pdf,
    create_text_image,
    get_all_image_bounding_boxes,
    test_image_placement,
    is_overlapping,
    append_image_to_pdf,
    random_augment
)

In [ ]:
import random
import numpy as np
from reportlab.lib.pagesizes import letter

width, height = int(letter[0]), int(letter[1])

text = "This is sample text for our PDF"
words_per_line = np.random.binomial(n=10, p=0.5, size=1)[0]+3
font_path = random.choice(system_fonts)
font_size = np.random.binomial(n=25,p=0.5,size=1)[0]+1
img, img_info = create_text_image(text, words_per_line=words_per_line, font_path=font_path, font_size=font_size)
img = random_augment(img)
pdf_path = "output.pdf"
canvas_obj, (page_width, page_height) = create_blank_pdf(pdf_path)
x_center, y_center = page_width/2, page_height/2
existing_boxes = []
if can_place_image(img, x_center, y_center, existing_boxes, page_width, page_height):
    place_image_in_pdf(canvas_obj, img, x_center, y_center)
else:
    position = find_optimal_placement(img, existing_boxes, page_width, page_height)
    if position:
        x_center, y_center = position
        place_image_in_pdf(canvas_obj, img, x_center, y_center)
    else:
        print("Cannot place image without overlaps")

In [4]:
new_text = "Marroons are cool and GPT is fun"
max_retries = 15
retry_count = 0
new_bbox = None

while retry_count < max_retries:
    new_words_per_line = np.random.binomial(n=10, p=0.5, size=1)[0]+3
    new_font_path = random.choice(system_fonts)
    new_font_size = np.random.binomial(n=25,p=0.5,size=1)[0]+1
    new_x_centroid = random.randint(0, int(page_width))
    new_y_centroid = random.randint(0, int(page_height))
    new_text_image, new_text_info = create_text_image(new_text, words_per_line=new_words_per_line, font_path=new_font_path, font_size=new_font_size)
    new_text_image = random_augment(new_text_image)
    x_center, y_center = page_width/2, page_height/2
    existing_boxes = []
    if can_place_image(new_text_image, new_x_centroid, new_y_centroid, existing_boxes, page_width, page_height):
        place_image_in_pdf(canvas_obj, new_text_image, new_x_centroid, new_y_centroid)
        break
    else:
        position = find_optimal_placement(new_text_image, existing_boxes, page_width, page_height)
        if position:
            x_center, y_center = position
            place_image_in_pdf(canvas_obj, new_text_image, x_center, y_center)
            break
        else:
            print("Cannot place image without overlaps")
    print(f"Overlap detected, retrying... ({retry_count + 1}/{max_retries})")
    retry_count += 1

if retry_count == max_retries:
    print("Failed to place new text without overlap after multiple attempts.")

save_pdf(canvas_obj)